In [ ]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras import optimizers
from keras.utils import np_utils
import keras.backend as K
from keras.utils import to_categorical
import keras
from tensorflow.keras.layers import BatchNormalization
import tensorflow.keras.metrics as metrics
import matplotlib.pyplot as plt

train_imgs = np.load('train_images.npy')
train_labels = pd.read_csv('train_labels.csv')
test_imgs = np.load('test_images.npy')

In [ ]:
unique_categories = np.unique(train_labels)

datasets = {}

for category in unique_categories:
    indices = np.where(train_labels == category)[0]
    category_images = train_imgs[indices]
    datasets[category] = category_images

print(datasets.keys())

In [1]:
def plot_histograms():
    fig, axes = plt.subplots(5, 2, figsize=(10, 12))

    keys = {
    0:    'T-shirt/top',
    1:    'Trouser',
    2:    'Pullover',
    3:    'Dress',
    4:    'Coat',
    5:    'Sandal',
    6:    'Shirt',
    7:    'Sneaker',
    8:    'Bag',
    9:    'Ankle boot'}

    for ind, ax in enumerate(axes.flatten()):
        counts, bins = np.histogram(datasets[ind].flatten())
        ax.stairs(counts/datasets[ind].flatten().size, bins)
        ax.set_ylim([0,.8])
        ax.set_title(str(keys[ind]))

    plt.tight_layout()
    plt.show()

In [3]:
def print_average_items():
    fig, axes = plt.subplots(2, 5)

    for ind, ax in enumerate(axes.flatten()):

        ax.imshow(np.average(datasets[ind], axis=0))
        ax.set_title(str(keys[ind]))

    fig.suptitle('Figuras promedio de las clases')
    fig.tight_layout()

    plt.show()

In [5]:

def create_softmax_model(output_size = 10):
    
    normal_initializer = keras.initializers.GlorotNormal
    uniform_initializer = keras.initializers.GlorotUniform
    K.clear_session()
    softmax_model = Sequential()
    softmax_model.add(Flatten(input_shape=(28,28)))
    softmax_model.add(BatchNormalization())
    softmax_model.add(Dense(output_size, activation='softmax',  kernel_initializer=uniform_initializer, name='salida'))
    softmax_model.summary()

    import time
    adam = optimizers.Adam(learning_rate=0.001, decay=1E-4) #decay=1e-4
    checkpointer = ModelCheckpoint(filepath='softmax.mnist.hdf5', verbose=1, save_best_only=True, mode='max', monitor='val_accuracy')

    earlyStopping = keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        min_delta=0,
        patience=5,
        verbose=0,
        mode="auto",
        baseline=None,
        restore_best_weights=True,
    )

    softmax_model.compile(loss = 'categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

    softmax_model.fit(X_train, 
            y_train_categorical ,
            epochs=100, batch_size=2048, 
            verbose=0, 
            validation_data=(X_val, y_val_categorical), 
            callbacks=[earlyStopping],
            )
    return softmax_model



In [ ]:
def train_model(X_train, y_train_categorical, X_val, y_val_categorical, opt, b_s, act, pat, init):

    earlyStopping = keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        min_delta=0,
        patience=pat,
        verbose=0,
        mode="auto",
        baseline=None,
        restore_best_weights=False,
    )

    output_size = 10

    K.clear_session()
    model_mpl = Sequential()
    model_mpl.add(Flatten(input_shape=(28,28)))
    model_mpl.add(BatchNormalization())
    model_mpl.add(Dense(500, activation=act, kernel_initializer=init, name='middle1'))
    model_mpl.add(BatchNormalization())
    model_mpl.add(Dense(200, activation=act, kernel_initializer=init, name='middle2'))
    model_mpl.add(Dense(output_size, activation='softmax',  kernel_initializer=init, name='salida'))
    model_mpl.compile(loss = 'categorical_crossentropy', optimizer=opt , metrics=['accuracy'])

    #sys.stdout = open(os.devnull, 'w')
    history = model_mpl.fit(X_train, 
        y_train_categorical ,
        epochs=100, batch_size=b_s, 
        verbose=0, 
        validation_data=(X_val, y_val_categorical), 
        callbacks=[earlyStopping],
        )

    val_acc = history.history["val_accuracy"]

    return val_acc, model_mpl

In [ ]:
def make_batch_size_graphs():
    plt.xlabel("batch size")
    plt.ylabel("validation accuracy")
    plt.title("Validation accuracy vs Batch Size")
    plt.plot(batch_sizes, [x[-1] for x in accuracy_values])
    plt.show()
    plt.xlabel("batch size")
    plt.ylabel("number of iterations")
    plt.title("Numer of Iterations vs Batch Size")
    plt.plot(batch_sizes, [len(x) for x in accuracy_values])

In [ ]:
def make_learning_rate_graphs():
    plt.xlabel("learning rate")
    plt.ylabel("validation accuracy")
    plt.xscale('log')
    plt.title('Val Accuracy vs Learning rate for SGD optimizer')
    plt.plot(learning_rates, [x[-1] for x in accuracy_values])
    plt.show()

    plt.xlabel("learning rate")
    plt.ylabel("validation accuracy")
    plt.xscale('log')
    plt.title('Numer of Iterations vs Learning rate for SGD optimizer')
    plt.plot(learning_rates, [len(x) for x in accuracy_values])

In [ ]:
def make_optimizer_graphs():
    plt.bar(['SGD', 'adam', 'adamax', 'nadam'], [x[-1] for x in accuracy_values], color ='maroon',
            width = 0.4)
    plt.xlabel("optimizer")
    plt.ylabel("validation accuracy")
    plt.title('Validation Accuracy vs Optimizer')
    plt.ylim(0.8, 0.9)
    plt.show()

    plt.bar(['SGD', 'adam', 'adamax', 'nadam'], [len(x) for x in accuracy_values], color ='maroon',
            width = 0.4)

    plt.xlabel("optimizer")
    plt.ylabel("validation accuracy")
    plt.title('Numer of Iterations vs Optimizer')

In [ ]:
def make_activation_graphs():
    # Create the bar plot
    # creating the bar plot
    plt.bar(activations, [x[-1] for x in accuracy_values], color ='maroon',
            width = 0.4)

    # Set labels and title
    plt.xlabel("activation")
    plt.ylabel("validation accuracy")
    plt.title('Validation Accuracy vs Activation Function')

    # Display the plot

    # Display the plot
    plt.ylim(0.8, 0.9)
    plt.show()

    # Create the bar plot
    # creating the bar plot
    plt.bar(activations, [len(x) for x in accuracy_values], color ='maroon',
            width = 0.4)

    # Set labels and title
    plt.xlabel("activation")
    plt.ylabel("validation accuracy")
    plt.title('Number of Iterations vs Activation Function')

In [ ]:
def make_initializer_graphs():
    # Create the bar plot
    # creating the bar plot
    plt.bar(['N sigma .1', 'N sigma 1', 'N Glorot', 'U Glorot'], [x[-1] for x in accuracy_values], color ='maroon',
            width = 0.4)

    # Set labels and title
    plt.xlabel("Kernel Initialization")
    plt.ylabel("validation accuracy")
    plt.title('Validation Accuracy vs Initialization')

    # Display the plot

    # Display the plot
    plt.ylim(0.84, 0.9)
    plt.show()

    # Create the bar plot
    # creating the bar plot
    plt.bar(['N sigma .1', 'N sigma 1', 'N Glorot', 'U Glorot'], [len(x) for x in accuracy_values], color ='maroon',
            width = 0.4)

    # Set labels and title
    plt.xlabel("Kernel Initialization")
    plt.ylabel("validation accuracy")
    plt.title('Number of Iterations vs Initialization')